<br>
<h1 style = "font-size:40px; font-family:Garamond ; font-weight : normal; background-color: #C66363 ; color : #E8D6D8; text-align: center; border-radius: 100px 100px;">CONTENT </h1>
<br>

* [Add Libaries](#1)
* [Preproccess](#2)
* [Prepare And Run Model](#3)
* [Visualize Model Results](#4)

<a id="1"> </a>
# Add Libaries


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa

from collections import Counter
from tensorflow import keras
from keras.models import Sequential,load_model
from keras.layers import Dense,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Dropout,MaxPooling2D,Activation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator as Imgen
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from sklearn.metrics import confusion_matrix,classification_report
import cv2
import os

import warnings
warnings.filterwarnings("ignore")

<a id="2"> </a>
# Preproccess


In [ ]:
train_df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
train_path = "../input/plant-pathology-2021-fgvc8/train_images"
test_path="../input/plant-pathology-2021-fgvc8/sample_submission.csv"
test = pd.read_csv(test_path)
test

In [ ]:
plt.figure(figsize=(15,30))
i=1
for idx,s in train_df.head(9).iterrows():
    img_path = os.path.join(train_path,s['image'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    fig=plt.subplot(9,3,i)
    fig.imshow(img)
    fig.set_title(s['labels'])
    i+=1

In [ ]:
train_df.head(9)

In [ ]:
train_df.info()

In [ ]:
labels = train_df["labels"].value_counts().index
data = train_df["labels"].value_counts().values

In [ ]:
print(labels)
print(data)

In [ ]:
class_num = len(train_df["labels"].value_counts())

In [ ]:
datagen = ImageDataGenerator(rescale=1/255.0,
                            rotation_range=5,
                            zoom_range=0.1,
                            shear_range=0.05,
                            horizontal_flip=True,
                            validation_split=0.2)

In [ ]:
train_ds = datagen.flow_from_dataframe(
    train_df,
    directory='../input/resized-plant2021/img_sz_256',
    subset='training',
    x_col='image',
    y_col='labels',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=444
    )
#'../input/plant-pathology-2021-fgvc8/train_images'
valid_ds = datagen.flow_from_dataframe(
    train_df,
    directory='../input/resized-plant2021/img_sz_256',
    subset='validation',
    x_col='image',
    y_col='labels',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=444
    )



In [ ]:
x,y = next(train_ds)
x.shape

<a id="3"> </a>
# Prepare And Run Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
vgg = VGG16()

In [ ]:
model = Sequential()
vgg_layers = vgg.layers

for i in range(len(vgg_layers)-1):
    model.add(vgg_layers[i])

In [ ]:
for l in vgg_layers:
    l.trainable = False

In [ ]:
model.add(Dense(class_num , activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
import tensorflow_addons as tfa
import keras as keras

f1 = tfa.metrics.F1Score(num_classes=6,average='macro')

metrics = [
    keras.metrics.CategoricalAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall')
    ] 

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=[metrics])
batch_size = 32

es=EarlyStopping(patience=4, monitor=f1, mode='max',restore_best_weights=True)
hist = model.fit_generator(generator=train_ds,
                           validation_data=valid_ds,
                           epochs=200,
                           steps_per_epoch=train_ds.samples//128,
                           validation_steps=valid_ds.samples//128,
                           callbacks=[es])

In [ ]:
test_data = datagen.flow_from_dataframe(
    test,
    directory='../input/plant-pathology-2021-fgvc8/test_images',
    x_col='image',
    y_col=None,
    color_mode='rgb',
    target_size=(224,224),
    class_mode=None,
    shuffle=False
)
predictions = model.predict(test_data)
print(predictions)


<a id="4"> </a>
# Visualize Model Results

In [ ]:
print(hist.history.keys())

print(hist.history['accuracy'])
print(hist.history['loss'])
print(hist.history['recall'])
print(hist.history['precision'])


In [ ]:
%matplotlib inline
plt.rc('font', size=20)
plt.figure(figsize=(15,12))
epoch_list = list(range(1, len(hist.history['accuracy']) + 1))
plt.plot(epoch_list, hist.history['accuracy'],label='accuracy')
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
plt.rc('font', size=20)
plt.figure(figsize=(15,12))
epoch_list = list(range(1, len(hist.history['loss']) + 1))
plt.plot(epoch_list, hist.history['loss'],label='loss')
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
plt.rc('font', size=20)
plt.figure(figsize=(15,12))
epoch_list = list(range(1, len(hist.history['recall']) + 1))
plt.plot(epoch_list, hist.history['recall'],label='recall')
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
plt.rc('font', size=20)
plt.figure(figsize=(15,12))
epoch_list = list(range(1, len(hist.history['precision']) + 1))
plt.plot(epoch_list, hist.history['precision'],label='precision')
plt.legend()
plt.show()

In [ ]:
#f1_score
"""plt.figure(figsize=(15,6))
epoch_list = list(range(1,len(hist.history['f1_score'])+1))
plt.plot(epoch_list, hist.history['f1_score'],label='f1_score')
plt.plot(epoch_list, hist.history['val_f1_score'],label='val_f1_score')
plt.xlabel('epoches')
plt.ylabel('f1')
plt.legend()
plt.show()"""

In [ ]:
#av
''''arr1 = hist.history['loss']
result1 = sum(arr1)
print(f"loss_av : {result1 / len(arr1)}")

arr2 = hist.history['accuracy']
result2 = sum(arr2)
print(f"accuracy_av : {result2 / len(arr2)}")

arr3 = hist.history['val_loss']
result3 = sum(arr3)
print(f"val_loss_av : {result3 / len(arr3)}")

arr4 = hist.history['val_accuracy']
result4 = sum(arr4)
print(f"val_accuracy_av : {result4 / len(arr4)}")'''